<a href="https://colab.research.google.com/github/erofale/encoderProject/blob/master/Code/Notebooks/enc_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from typing import Tuple, List


''' Класс нормировки данных '''
class Normalizer():
    
    '''
    Parameters
    ----------
    dim : int
        Размерность входных данных.
    range_val_pairs : List[Tuple[float,float]]
        Диапазон значений входных данных.
    norm_min : float, optional
        Нижняя граница нормированных данных (по умолчанию 0).
    norm_max : float, optional
        Верхняя граница нормированных данных (по умолчанию 1).
    '''
    def __init__(self, dim : int, range_val_pairs : List[Tuple[float,float]], norm_min : float = 0., norm_max : float = 1.):
        self.dim = dim
        self.range_val_pairs = range_val_pairs
        self.range_val = [(i[1] - i[0]) for i in self.range_val_pairs]
        self.norm_min = norm_min
        self.norm_max = norm_max
        self.range_norm = norm_max - norm_min
    
    
    def __normire(self, entryVal : float, ind : int) -> float:
        '''
        Parameters
        ----------
        entryVal : float
            Входное значение для нормировки.
        ind : int
            Индекс элемента в массиве.

        Returns
        -------
        float
            Нормированное значение.

        '''
        return self.norm_min + ((entryVal - self.range_val_pairs[ind][0]) * self.range_norm / self.range_val[ind])
    
    def __renormire(self, normVal : float, ind : int) -> float:
        '''
        Parameters
        ----------
        normVal : float
            Нормированное значение для денормировки.
        ind : int
            Индекс элемента в массиве.

        Returns
        -------
        float
            Денормированное значение.
        '''
        return self.range_val_pairs[ind][0] + ((normVal - self.norm_min) / self.range_norm * self.range_val[ind])
    
    def normalize(self, data) -> List[List[float]]:
        '''
        Parameters
        ----------
        data : list, array
            Входной набор данных для нормировки.

        Returns
        -------
        List[List[float]]
            Нормированный набор данных.
        '''
        count = 0
        if type(data) == list:
            count = len(data)
        else:
            count = data.shape[0]
        normData = []
        for i in range(count):
            cur_sample = []
            for j in range(self.dim):
                cur_sample.append(self.__normire(data[i][j], j))
            normData.append(cur_sample)
        return normData
    
    def renormalize(self, normData) -> List[List[float]]:
        '''
        Parameters
        ----------
        normData : list, array
            Нормированный набор данных.

        Returns
        -------
        List[List[float]]
            Денормированный набор данных.
        '''
        count = 0
        if type(normData) == list:
            count = len(normData)
        else:
            count = normData.shape[0]
        data = []
        for i in range(count):
            cur_sample = []
            for j in range(self.dim):
                cur_sample.append(self.__renormire(normData[i][j], j))
            data.append(cur_sample)
        return data


In [12]:
import numpy as np
import random
import pandas as pd
#!pip install git+https://github.com/naught101/sobol_seq@v0.2.0#egg=sobol_seq
import sobol_seq
import time
from typing import Tuple, List


''' Класс генерации данных '''
class DataGenerator():

    '''
    Parameters
    ----------
    dim: int
        Размерность входных данных.
    val_range : List[Tuple[float,float]]
        Диапазон значений входных данных.
    '''
    def __init__(self, dim : int, val_range : List[Tuple[float,float]]):
        try:
            assert dim  == len(val_range), 'Размерность входных диапазонов не равна входной размерности!'
            self.dim = dim
            self.val_range = val_range
            random.seed(int(time.time()))
        except AssertionError as e:
            raise AssertionError(e.args[0])
    

    def get_random(self, samples_num : int, irrelevant_var_count : int = 0, write_in_file : bool = False) -> List[List[float]]:
        '''
        Рандомная генерация данных.
        Parameters
        ----------
        samples_num : int
            Количество записей.
        irrelevant_var_count : int, optional
            Количество незначащих переменных. По умолчанию 0.
        write_in_file : bool, optional
            Запись в файл. По умолчанию False.

        Returns
        -------
        List[List[float]]
            Список записей.
        '''
        arr = []
        for k in range(samples_num):
            sample = []
            # добавляем существенные переменные
            for i in self.val_range:
                sample.append(random.uniform(i[0], i[1]))
            # добавляем несущественные переменные
            if irrelevant_var_count != 0:
                for i in range(irrelevant_var_count):
                    sample.append(random.uniform(0., 1.))
            arr.append(sample)
        if write_in_file:
            col = [('x' + str(i+1)) for i in range(self.dim + irrelevant_var_count)]
            df = pd.DataFrame(arr, columns=col)
            df.to_csv(f'../../DataSet/random_{self.dim}_{samples_num}_{irrelevant_var_count}.csv')
        return arr
    

    def get_sobol(self, samples_num : int, irrelevant_var_count : int = 0, write_in_file : bool = False) -> List[List[float]]:
        '''
        Генерация данных от 0 до 1 методом Sobol.
        Parameters
        ----------
        samples_num : int
            Количество записей.
        irrelevant_var_count : int, optional
            Количество незначащих переменных. По умолчанию 0.
        write_in_file : bool, optional
            Запись в файл. По умолчанию False.

        Returns
        -------
        List[List[float]]
            Список записей.
        '''
        arr = sobol_seq.i4_sobol_generate(self.dim, samples_num)
        if irrelevant_var_count != 0:
            zeros = [[0] for i in range(irrelevant_var_count)]
            arr = np.insert(arr, obj=self.dim, values=zeros, axis=1)
        if write_in_file:
            col = [('x' + str(i+1)) for i in range(self.dim + irrelevant_var_count)]
            df = pd.DataFrame(arr, columns=col)
            df.to_csv(f'../DataSet/sobol_{self.dim}_{samples_num}_{irrelevant_var_count}.csv')
        return list(arr)
    
    
    def get_from_file(self, filename : str) -> List[List[float]]:
        '''
        Parameters
        ----------
        filename : str
            Имя файла.

        Raises
        ------
        OSError
            Файл не найден.

        Returns
        -------
        List[List[float]]
            Список записей.
        '''
        try:
            return list(pd.read_csv(filename, index_col=0).to_numpy('float32'))
        except OSError as e:
            raise OSError(e.args[0])

In [13]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
import keras.backend as K
from keras.layers import Lambda
import numpy as np

''' Класс автоэнкодеров '''
class AutoencoderClass():
  def __init__(self, func, input_dim : int, encoding_dim : int, activations : list, enc_type : str, normalizer : Normalizer):
    self.func = func                 # Функция обучения
    self.batch = 0
    self.input_dim = input_dim       # Размерность входного представления
    self.encoding_dim = encoding_dim # Размерность кодированного представления
    self.activations = activations   # Функции активации
    self.enc_type = enc_type         # Тип автоэнкодера
    self.aes_types = {'dense': self.__create_dense_ae,
                      'deep':  self.__create_deep_dense_ae,
                      'conv':  self.__create_deep_conv_ae,
                      'vae':   self.__create_vae}
    self.normalizer = normalizer
    try:
      # Сборка моделей
      self.encoder, self.decoder, self.autoencoder = self.aes_types[self.enc_type]()
      self.autoencoder.compile(optimizer = 'adam', loss = self.custom_loss, metrics=['accuracy'])
    except KeyError as e:
      raise ValueError('Undefined unit: {}'.format(e.args[0]))

  # Обучение модели
  def fit(self, train_data, test_data, epochs : int, batch_size : int, shuffle : bool):
    self.batch = batch_size
    if self.enc_type != 'conv':
      self.autoencoder.fit(train_data, train_data,
                           epochs=epochs,
                           batch_size=self.batch,
                           shuffle=shuffle,
                           validation_data=(test_data, test_data))
    else:
      grid_train = []
      grid_test = []
      for i in range(len(train_data)):
        xx, yy = np.meshgrid(train_data[i], train_data[i])
        grid_train.append(xx)

      for i in range(len(test_data)):
        xx, yy = np.meshgrid(test_data[i], test_data[i])
        grid_test.append(xx)
      
      self.autoencoder.fit(grid_train, grid_train,
                           epochs=epochs,
                           batch_size=self.batch,
                           shuffle=shuffle,
                           validation_data=(grid_test, grid_test))

  # Предсказание результата
  def predict(self, x_vector):
    if self.enc_type != 'conv':
      return self.autoencoder.predict(x_vector)
    else:
      return self.autoencoder.predict(x_vector)[0]

  # Тип автоэнкодера
  def get_aec_type(self):
    return self.enc_type

  # Возвращает собранные модели
  def get_models(self):
    return self.autoencoder, self.encoder, self.decoder

  # Loss функция
  @tf.autograph.experimental.do_not_convert
  def custom_loss(self, x_true, x_pred):
    return K.mean(K.square(self.func(self.normalizer.renormalize([x_pred])[0]) - self.func(self.normalizer.renormalize([x_true])[0])))

  ''' Сжимающий автоэнкодер '''
  def __create_dense_ae(self):
    # Энкодер
    input_data = Input(shape=(self.input_dim))
    encoded = Dense(self.encoding_dim, activation = self.activations[0])(input_data)
    
    # Декодер
    input_encoded = Input(shape = (self.encoding_dim))
    decoded = Dense(self.input_dim, activation = self.activations[1])(input_encoded)

    # Модели
    encoder = Model(input_data, encoded, name = "encoder")
    decoder = Model(input_encoded, decoded, name = "decoder")
    autoencoder = Model(input_data, decoder(encoder(input_data)), name = "autoencoder")
    return encoder, decoder, autoencoder

  ''' Глубокий автоэнкодер '''
  def __create_deep_dense_ae(self):
    # Энкодер
    input_data = Input(shape=(self.input_dim))
    ############# Здесь фиксить надо с размерностями слоёв и как подавать активации на вход##########
    x = Dense(self.encoding_dim*3, activation='relu')(input_data)
    x = Dense(self.encoding_dim*2, activation='relu')(x)
    encoded = Dense(self.encoding_dim, activation='linear')(x)
    
    # Декодер
    input_encoded = Input(shape=(self.encoding_dim,))
    ############# Здесь фиксить надо с размерностями слоёв ##########
    x = Dense(self.encoding_dim*2, activation='relu')(input_encoded)
    x = Dense(self.encoding_dim*3, activation='relu')(x)
    decoded = Dense(self.input_dim, activation='sigmoid')(x)
    
    # Модели
    encoder = Model(input_data, encoded, name="encoder")
    decoder = Model(input_encoded, decoded, name="decoder")
    autoencoder = Model(input_data, decoder(encoder(input_data)), name="autoencoder")
    return encoder, decoder, autoencoder

  ''' Сверточный автоэнкодер '''
  def __create_deep_conv_ae(self):
    # Энкодер
    input_data = Input(shape=(self.input_dim, self.input_dim, 1))
    x = Conv2D(25, (2, 2), activation='relu', padding='same')(input_data)
    x = MaxPooling2D((2, 2), padding='same')(x)
    #x = Conv2D(32, (2, 2), activation='relu', padding='same')(x)
    #x = MaxPooling2D((2, 2), padding='same')(x)
    encoded = Conv2D(1, (2, 2), activation='relu', padding='same')(x)

    # На этом моменте представление  (7, 7, 1) т.е. 49-размерное
    
    # Декодер
    input_encoded = Input(shape=(7, 7, 1))
    #x = Conv2D(32, (7, 7), activation='relu', padding='same')(input_encoded)
    #x = UpSampling2D((2, 2))(x)
    x = Conv2D(25, (2, 2), activation='relu', padding='same')(input_encoded)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (2, 2), activation='sigmoid', padding='same')(x)

    # Модели
    encoder = Model(input_data, encoded, name="encoder")
    decoder = Model(input_encoded, decoded, name="decoder")
    autoencoder = Model(input_data, decoder(encoder(input_data)), name="autoencoder")
    return encoder, decoder, autoencoder

  ''' Вариационный автоэнкодер                         '''
  ''' Работает на основе девергенции Кульбака-Лейблера '''
  ''' Идея: переход данных скрытого слоя к нормальному распределению'''
  ''' Статья: https://habr.com/ru/post/484756/ '''
  ''' Видео:  https://youtu.be/ebI3JLAcWqQ '''
  def __create_vae(self):
    hidden_dim = 2
    
    # Выключение нейронов во избежании переобучения
    def dropout_and_batch(x):
      return Dropout(0.3)(BatchNormalization()(x))

    input_data = Input(shape=(self.input_dim))
    x = Dense(self.encoding_dim, activation='relu')(input_data)
    x = dropout_and_batch(x)
    #x = Dense(128, activation='relu')(x)
    #x = dropout_and_batch(x)
    
    z_mean = Dense(self.encoding_dim)(x)    # Мат ожидание
    z_log_var = Dense(self.encoding_dim)(x) # Логарифм дисперсии
    
    # Нормальное распределение N(0, 1)
    def noiser(args):
      global z_mean, z_log_var
      z_mean, z_log_var = args
      N = K.random_normal(shape=(self.batch, self.encoding_dim), mean=0., stddev=1.0)
      return K.exp(z_log_var / 2) * N + z_mean
    
    # Преобразование данных в нормальное распределения
    h = Lambda(noiser, output_shape=(self.encoding_dim))([z_mean, z_log_var])
    
    input_encoded = Input(shape=(self.encoding_dim))
    d = Dense(self.encoding_dim, activation='relu')(input_encoded)
    d = dropout_and_batch(d)
    #d = Dense(256, activation='relu')(d)
    #d = dropout_and_batch(d)
    decoded = Dense(self.input_dim, activation='sigmoid')(d)
    
    encoder = Model(input_data, h, name='encoder')
    decoder = Model(input_encoded, decoded, name='decoder')
    vae = Model(input_data, decoder(encoder(input_data)), name="vae")
    return encoder, decoder, vae

In [11]:
import numpy as np
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

def func(x):
  return x[0]*x[1] + x[1]*x[2] + x[3]*x[3]


def compare(orig_data, pred_data):
    # clf = svm.SVC(kernel='linear', C=1, random_state=42)
    # scores_x = cross_val_score(clf, orig_data[0:10], pred_data[0:10], cv=5)
    # scores_y = cross_val_score(clf, [func(x) for x in orig_data][0:10], [func(x) for x in pred_data][0:10], cv=5)
    
    y_error = mean_squared_error(orig_data, pred_data)
    x_error = mean_squared_error([func(x) for x in orig_data], [func(x) for x in pred_data])
    
    print(f'Error X: {x_error:.2f}')
    print(f'Error Y: {y_error:.2f}')


if __name__ == "__main__":
    dim = 4
    irr_dim = 2
    data_range = [(0, 100), (0, 100), (0, 100), (0, 100)]
    generator = DataGenerator(dim, data_range)
    normalizer = Normalizer(dim, data_range)
    
    n = 30000
    sobol_data = generator.get_sobol(n, irr_dim)
    data_train = np.array(sobol_data[0:int(n * 0.7)])
    data_test = np.array(sobol_data[int(n * 0.7):n])
    
    model = AutoencoderClass(func, dim + irr_dim, 4, list(['relu', 'sigmoid']), 'dense', normalizer)
    model.fit(data_train, data_test, 30, 16, True)
    
    pred_data = normalizer.renormalize([model.predict(x.reshape(1,dim + irr_dim))[0] for x in sobol_data[0:100]])
    compare(normalizer.renormalize(sobol_data)[0:100], pred_data[0:100])

Epoch 1/30
1313/1313 [==============================] - 3s 2ms/step - loss: 21642148.0000 - accuracy: 0.3061 - val_loss: 11262347.0000 - val_accuracy: 0.4117
Epoch 2/30
1313/1313 [==============================] - 3s 2ms/step - loss: 13483430.0000 - accuracy: 0.5266 - val_loss: 8895773.0000 - val_accuracy: 0.5412
Epoch 3/30
1313/1313 [==============================] - 3s 2ms/step - loss: 11558032.0000 - accuracy: 0.5299 - val_loss: 7880265.5000 - val_accuracy: 0.5207
Epoch 4/30
1313/1313 [==============================] - 2s 2ms/step - loss: 10293317.0000 - accuracy: 0.5376 - val_loss: 6616661.0000 - val_accuracy: 0.5284
Epoch 5/30
1313/1313 [==============================] - 2s 2ms/step - loss: 9438550.0000 - accuracy: 0.5517 - val_loss: 5621153.5000 - val_accuracy: 0.5549
Epoch 6/30
1313/1313 [==============================] - 3s 2ms/step - loss: 8611608.0000 - accuracy: 0.5720 - val_loss: 4719421.0000 - val_accuracy: 0.5816
Epoch 7/30
1313/1313 [==============================] - 2s 